In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import sys

# Add the project root to sys.path
project_root = os.path.abspath("..")  # one level up from 'notebooks'
sys.path.append(project_root)


Feature parent programme

In [ ]:

# Load the JSON file
df = pd.read_json("../Data/programme.json")

# Display unique values in the 'parentProgramme' column
unique_values = df["parentProgramme"].unique()

# Print results
print("Unique values in 'parentProgramme':")
for value in unique_values:
    print(value)


Unique values in 'parentProgramme':
HORIZON.3.2
HORIZON.3
HORIZON.1.3
HORIZON.1
HORIZON.3.1
HORIZON.2.1
HORIZON.2.2
HORIZON.2
HORIZON.2.4
HORIZON.2.3
HORIZON.2.6
HORIZON.2.5
HORIZON.4.2
HORIZON.1.2
HORIZON.4

HORIZON.1.1
EURATOM2027
HORIZON
EURATOM.1.1
HORIZON.4.1
EURATOM.1.2
EURATOM.1.3
EURATOM.1
HORIZON.2.7
EURATOM-1
EURATOM.1.4


In [6]:
#Unique values in legal basis

# Load the JSON file
df = pd.read_json("../Data/project.json")

# Display unique values in the 'parentProgramme' column
unique_values = df["legalBasis"].unique()

# Print results
print("Unique values in 'legalBasis':")
for value in unique_values:
    print(value)


Unique values in 'legalBasis':
HORIZON.3.1
HORIZON.2.1
HORIZON.2.6
HORIZON.2.3
HORIZON.3.2
HORIZON.2.4
HORIZON.4.1
HORIZON.1.3
HORIZON.2.2
HORIZON.2.5
EURATOM2027
HORIZON.1.2
HORIZON.4.2
HORIZON.1.1
HORIZON.3.3


Creating base version of selected_features

In [ ]:
# Load project data
df = pd.read_json("../Data/project.json")

# Convert date columns to datetime
df["startDate"] = pd.to_datetime(df["startDate"], errors="coerce")
df["endDate"] = pd.to_datetime(df["endDate"], errors="coerce")
df["ecSignatureDate"] = pd.to_datetime(df["ecSignatureDate"], errors="coerce")

# Compute derived features
df["startupDelay"] = (df["startDate"] - df["ecSignatureDate"]).dt.days
df["duration"] = (df["endDate"] - df["startDate"]).dt.days
df["totalCostzero"] = df["totalCost"].fillna(0).eq(0).astype(int)
df["contRatio"] = df.apply(
    lambda row: row["ecMaxContribution"] / row["totalCost"]
    if pd.notnull(row["ecMaxContribution"]) and pd.notnull(row["totalCost"]) and row["totalCost"] != 0
    else None,
    axis=1
)

# Legal basis to pillar mapping
mapping = {
    "HORIZON.1.1": "Pillar 1 - European Research Council (ERC)",
    "HORIZON.1.2": "Pillar 1 - Marie Sklodowska-Curie Actions (MSCA)",
    "HORIZON.1.3": "Pillar 1 - Research infrastructures",
    "HORIZON.2.1": "Pillar 2 - Health",
    "HORIZON.2.2": "Pillar 2 - Culture, creativity and inclusive society",
    "HORIZON.2.3": "Pillar 2 - Civil Security for Society",
    "HORIZON.2.4": "Pillar 2 - Digital, Industry and Space",
    "HORIZON.2.5": "Pillar 2 - Climate, Energy and Mobility",
    "HORIZON.2.6": "Pillar 2 - Food, Bioeconomy Natural Resources, Agriculture and Environment",
    "HORIZON.3.1": "Pillar 3 - The European Innovation Council (EIC)",
    "HORIZON.3.2": "Pillar 3 - European innovation ecosystems",
    "HORIZON.3.3": "Pillar 3 - Cross-cutting call topics",
    "EURATOM2027": "EURATOM2027",
    "EURATOM.1.1": "Improve and support nuclear safety, security, safeguards, radiation protection, safe spent fuel and radioactive waste management and decommissioning, including the safe and secure use of nuclear power and of non-power applications of ionizing radiation",
    "EURATOM.1.2": "Maintain and further develop expertise and competence in the nuclear field within the Community",
    "EURATOM.1.3": "Foster the development of fusion energy and contribute to the implementation of the European fusion roadmap",
}

df["pillar"] = df["legalBasis"].map(mapping)

# Select relevant columns
selected = df[[
    "id", "startupDelay", "totalCost", "totalCostzero",
    "ecMaxContribution", "duration", "contRatio", "pillar"
]]

# Drop rows where startupDelay is missing (optional, depending on use case)
selected = selected.dropna(subset=["startupDelay"])

# Convert to list of dicts and save as JSON
output_path = "../Data/selected_features.json"
selected.to_json(output_path, orient="records", indent=2, date_format="iso")

print(f"Selected features saved to {output_path}")

Selected features saved to ../Data/selected_features.json


Enriching selected_features with features from organization.json

In [3]:
# Load files
selected_df = pd.read_json("../Data/selected_features.json")
org_df = pd.read_json("../Data/organization.json")

# Ensure 'id' and 'projectID' are treated consistently
org_df["projectID"] = org_df["projectID"].astype(str)
selected_df["id"] = selected_df["id"].astype(str)

# --- Feature 1: countryCoor ---
coor_map = org_df[org_df["role"] == "coordinator"][["projectID", "country"]]
coor_map = coor_map.drop_duplicates("projectID").set_index("projectID")["country"]

# --- Feature 2: numberOrg ---
number_org = org_df.groupby("projectID").size()

# --- Feature 3: numberCountry ---
number_country = org_df.groupby("projectID")["country"].nunique()

# Merge into selected_df
selected_df["countryCoor"] = selected_df["id"].map(coor_map)
selected_df["numberOrg"] = selected_df["id"].map(number_org).fillna(0).astype(int)
selected_df["numberCountry"] = selected_df["id"].map(number_country).fillna(0).astype(int)

# Save the enriched file
output_path = "../Data/selected_features.json"
selected_df.to_json(output_path, orient="records", indent=2)

print(f"Enriched selected_features.json saved to {output_path}")

Enriched selected_features.json saved to ../Data/selected_features.json


Dealing with negative startup delays and totalCost being zero

In [4]:
# Load the JSON file
df = pd.read_json("../Data/selected_features.json")

# Count the number of entries (rows)
num_entries = len(df)

print(f"Number of entries in selected_features.json: {num_entries}")

Number of entries in selected_features.json: 16914


In [5]:

# Load the JSON file
df = pd.read_json("../Data/selected_features.json")

# Filter out rows with negative startupDelay
df_cleaned = df[df["startupDelay"] >= 0]

# Save the cleaned dataset back to the same file
df_cleaned.to_json("../Data/selected_features.json", orient="records", indent=2)

print(f"Removed entries with negative startupDelay. Remaining entries: {len(df_cleaned)}")

Removed entries with negative startupDelay. Remaining entries: 16113


Dealing with categorical variables with multiple levels - exploring options

In [6]:


# Load the data
df = pd.read_json("../Data/selected_features.json")

# Frequency counts for 'pillar'
print("Value counts for 'pillar':")
print(df["pillar"].value_counts(dropna=False))
print("\n")

# Frequency counts for 'countryCoor'
print("Value counts for 'countryCoor':")
print(df["countryCoor"].value_counts(dropna=False))


Value counts for 'pillar':
pillar
Pillar 1 - Marie Skłodowska-Curie Actions (MSCA)                              5858
Pillar 1 - European Research Council (ERC)                                    4562
Pillar 2 - Climate, Energy and Mobility                                       1085
Pillar 2 - Digital, Industry and Space                                        1013
Pillar 3 - The European Innovation Council (EIC)                               838
Pillar 2 - Food, Bioeconomy Natural Resources, Agriculture and Environment     678
None                                                                           652
Pillar 2 - Health                                                              493
Pillar 2 - Culture, creativity and inclusive society                           316
Pillar 3 - European innovation ecosystems                                      264
Pillar 1 - Research infrastructures                                            158
Pillar 2 - Civil Security for Society                

In [ ]:
#TotalCost being zero

In [7]:
import pandas as pd

# Load the dataset
df = pd.read_json("../Data/selected_features.json")

# Count entries with totalCost == 0
zero_total_cost_count = (df["totalCost"] == 0).sum()

print(f"Number of entries with totalCost = 0: {zero_total_cost_count}")

Number of entries with totalCost = 0: 7780


Dropping numberCountry given strong correlation revealed during pre-processing

In [3]:
df = pd.read_json("../Data/selected_features.json")

# Drop the 'numberCountry' column if it exists
if "numberCountry" in df.columns:
    df = df.drop(columns=["numberCountry"])
    print("✅ Dropped 'numberCountry' column.")
else:
    print("ℹ️ Column 'numberCountry' not found — nothing to drop.")

# Save the updated dataset
df.to_json("../Data/selected_features.json", orient="records", indent=2)
print("✅ Updated file saved to ../Data/selected_features.json")

✅ Dropped 'numberCountry' column.
✅ Updated file saved to ../Data/selected_features.json
